## Site Access Score Tool

--------------------------------------------------------------------------------------------


#### Round 1

* GetParameterAsText (https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/getparameterastext.htm)
* SpatialReference (https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/spatialreference.htm)
* PointGeometry (https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/pointgeometry.htm)
* Copy Features (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/copy-features.htm)
* Buffer Analysis (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/buffer.htm)
* Intersect Analysis (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/intersect.htm)
* os.path.join() (https://www.geeksforgeeks.org/python-os-path-join-method/)

1) Created a point with longitude, latitude number (double) <br>
2) Created a buffer with a customized distance (linear unit) <br> 
3) Created point feature layers intersected between the buffer and major roads <br> 


--------------------------------------------------------------------------------------------


#### Round 2

* OriginDestinationCostMatrix (https://pro.arcgis.com/en/pro-app/2.8/arcpy/network-analyst/odcostmatrix.htm)
* Join Field (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/join-field.htm)
* Copy (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/copy.htm)
* Table to Excel (Conversion) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/conversion/table-to-excel.htm)
* Summary Statistics (Analysis) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/summary-statistics.htm)
* UpdateCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/updatecursor-class.htm)
  * SearchCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/searchcursor-class.htm)
  * InsertCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/insertcursor-class.htm)
* AddMessage (https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/addmessage.htm)
* Delete (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/delete.htm)

1) Added workspace as a parameter for different environment settings <br>
2) Created drive times(minutes) and network distance(miles) from buffered points intersected with major roads to a target point <br>
3) Used our Network Dataset, not esri's one <br>
4) Exported excel(xlsx) files for the OD Cost Matrix as well as the summary calculation. <br>
5) Added each point's drive time and network distance to the result message. <br>


--------------------------------------------------------------------------------------------


#### Round 3

* Iteraiting Output Names on Feature Classes with row values in a table (https://gis.stackexchange.com/questions/206401/making-lists-looping-a-function-through-a-list-and-controlling-output-names)
* Choosing between the Modules (arcpy.na and arcpy.nax) (https://pro.arcgis.com/en/pro-app/latest/arcpy/network-analyst/choosing-between-the-two-modules-arcpy-nax-versus-arcpy-na-.htm)
* Layer File (.lyr) and Lyaers File (.lyrx) (https://pro.arcgis.com/en/pro-app/2.8/arcpy/mapping/layerfile.htm#:~:text=lyr%20files%20only%20store%20a,layers%20at%20the%20root%20level.)


1) Multiple Input Locations & Buffer Distance with csv file - Used csv as an inpute parameter with fields, which are "ID", "Name", "Lat", "Long", "Buffer_Distance" <br>
2) After creating multiple feature layers, combine them as one layer by location and buffer <br>
3) Analyze OD Cost Matrix for all locations and set the destination as 1. <br>
4) Create a summary table by locations <br>
5) Research the drive time of day - UTC / Local Time
                 Chicago  LA      New York  
  * UTC, 14:30 - 4.656    2.217   4.7377
  * UTC, 15:00 - 4.686    2.334   4.8135
  * UTC, 15:30 - 4.675    2.374   4.8109 <br>

6) Set the time of day as an Input Parameter for the drvie time analysis <br>
7) Created the tools with three different ways, which are ODCostMatrix, Make Closest Facility Analysis Layer, and Closest Facility. <br>
8) Found that you can create network analysis layers in the arcpy.na module; but you cannot in the arcpy.nax module. <br>

>| Note |
<br>
* June 1st, 2022
  * Use For Loop to analyze multiple locations with customized buffer distance at once
  * Change input parameter from lat, long to csv 
* June 2nd, 2022
  * Add customized drive time analysis based on time of day to produce better outcome
* June 3rd, 2022
  * Create 2nd version SAS tool with Closes Facility tool
>


7) Calculate the number of turns (right or left) <br>
>| Tips |
<br>
* Copy Traversed Source Features (https://desktop.arcgis.com/en/arcmap/latest/tools/network-analyst-toolbox/copy-traversed-source-features.htm)
>



1. Driving Time is not ideal values
  * Current drive time is similar the drive time at 6:00 AM
  * Actual Drive time should be higher than current values
    * => Use the Jeff's number to make the number reasonable
<br>
<br>

In [ ]:
### SAS - 220606 - v3-3).Closest Facility + Copy Traversed Source Features

# import system modules
import arcpy
from arcpy import env
import os
import datetime
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
DepartureTime = arcpy.GetParameterAsText(2) # Date (Date & Time)
output_table_xlsx = arcpy.GetParameterAsText(3) # xlsx
output_summary_xlsx = arcpy.GetParameterAsText(4) # xlsx

# define workspace & create feature dataset for SAS feature layers
arcpy.env.workspace = work_dbs
workspace = work_dbs
arcpy.management.CreateFeatureDataset(out_dataset_path = workspace, 
                                      out_name = "SAS",
                                      spatial_reference = arcpy.SpatialReference("WGS 1984")
                                     )

# For each row, print the fields of feature class
fields = ['ID', 'Lat', 'Long', 'Buffer_Distance']
sr = arcpy.SpatialReference("WGS 1984")
with arcpy.da.SearchCursor(loc_csv, fields) as cursor:
    for row in cursor:
        pointNum = 'point_{}'.format(row[0])
        pointGeom_Num = 'pointGeom_{}'.format(row[0])
        pointFlyr_Num = 'pointFlyr_{}'.format(row[0])
        pointBuff_Num = 'pointBuff_{}'.format(row[0])
        pointBuff_Dis = '{} Miles'.format(row[3])
        pointNum = arcpy.Point(row[2],row[1])
        pointGeom_Num = arcpy.PointGeometry(pointNum,sr)
        arcpy.CopyFeatures_management(pointGeom_Num, pointFlyr_Num)
        arcpy.Buffer_analysis(pointFlyr_Num, pointBuff_Num, pointBuff_Dis)

# List all of the polygon feature classes that start with pointFlyr and pointBuff
list_pFl = arcpy.ListFeatureClasses("pointFlyr*", "point")        
list_pBf = arcpy.ListFeatureClasses("pointBuff*", "Polygon")
# merge all of the point and polygon features 
merged_point = os.path.join(workspace, "merged_point")
merged_polygon = os.path.join(workspace, "merged_polygon")
arcpy.Merge_management(list_pFl, merged_point)
arcpy.Merge_management(list_pBf, merged_polygon)

# Copy and join the fields from the csv file
point_merged = os.path.join(workspace, "SAS", "Target_point")
polygon_merged = os.path.join(workspace, "SAS", "Area_polygon")
arcpy.JoinField_management(merged_point, "OBJECTID", 
                           loc_csv, "ID")
arcpy.JoinField_management(merged_polygon, "OBJECTID", 
                           loc_csv, "ID")
arcpy.Copy_management(merged_point, point_merged)
arcpy.Copy_management(merged_polygon, polygon_merged)

# create intersections between the buffer and major roads feature layer
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "SAS", "Road_point")
arcpy.Intersect_analysis([polygon_merged, major_roads], intsct_point, "", "", "point")

# 2. Network Analysis - Closest Facility with arcpy.nax (arcpy.nax cannot create a network analysis)
# set local variables to make closest facility analysis layer and solve it.
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_facilities = point_merged
input_incidents = intsct_point
output_routes = os.path.join(workspace, "SAS", "Route_line")

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a ClosestFacility solver object
closest_facility = arcpy.nax.ClosestFacility(nds)
# Set properties
closest_facility.distanceUnits = arcpy.nax.DistanceUnits.Miles
closest_facility.routeShapeType = arcpy.nax.RouteShapeType.TrueShapeWithMeasures
closest_facility.timeOfDay = datetime.datetime.strptime(DepartureTime, '%m/%d/%Y %I:%M:%S %p')
closest_facility.timeOfDayUsage = arcpy.nax.TimeOfDayUsage.DepartureTime
closest_facility.timeUnits = arcpy.nax.TimeUnits.Minutes
closest_facility.timeZone = arcpy.nax.TimeZoneUsage.UTC
closest_facility.travelDirection = arcpy.nax.TravelDirection.ToFacility
closest_facility.travelMode = travel_mode
closest_facility.defaultTargetFacilityCount = 1
# Load inputs
closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Facilities, input_facilities)
closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Incidents, input_incidents)

# Solve the analysis
result = closest_facility.solve()

# Export the results to a feature class
if result.solveSucceeded:
    result.export(arcpy.nax.ClosestFacilityOutputDataType.Routes, output_routes)
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_routes, output_table_xlsx)

# Execute Summary Statistics and create a table
output_summary = os.path.join(workspace, "Summary_table")
arcpy.Statistics_analysis(
    in_table = output_routes, 
    out_table = output_summary, 
    statistics_fields = [["Total_Minutes", "MEAN"],
                         ["Total_Minutes", "MAX"],
                         ["Total_Minutes", "MIN"],
                         ["Total_Minutes", "MEDIAN"],
                         ["Total_Minutes", "RANGE"],
                         ["Total_Miles", "MEAN"],
                         ["Total_Miles", "MAX"],
                         ["Total_Miles", "MIN"],
                         ["Total_Miles", "MEDIAN"],
                         ["Total_Miles", "RANGE"],
                        ],
    case_field = "Name"
)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_summary, output_summary_xlsx)

# 3. Network Analysis - Make Closest Facility Analysis Layer with arcpy.na
# Create a new network analysis layer with Arcpy.na to calculate the driving directions
result_object = arcpy.na.MakeClosestFacilityAnalysisLayer(
    network_data_source = nds, 
    layer_name = "ClosestTarget",
    travel_mode = "Driving Time", 
    travel_direction = "TO_FACILITIES",
    number_of_facilities_to_find = 1, 
    time_of_day = DepartureTime, 
    time_zone = "UTC", 
    time_of_day_usage = "START_TIME",
    line_shape = "ALONG_NETWORK", 
    generate_directions_on_solve = False
)

# Get the layer object from the result object. The closest facility layer can now be referenced using the layer object.
layer_object = result_object.getOutput(0)

# Get the names of all the sublayers within the closest facility layer.
sublayer_names = arcpy.na.GetNAClassNames(layer_object)
# Stores the layer names that will be used later
facilities_layer_name = sublayer_names["Facilities"]
incidents_layer_name = sublayer_names["Incidents"]

# Laod the targets as Facilities using the default field mappings and search tolerance
arcpy.na.AddLocations(
    in_network_analysis_layer = layer_object,
    sub_layer = facilities_layer_name,
    in_table = point_merged)

# Load the starting major road points as Incidents. Map the Name property from the Name field using field mappings.
field_mappings = arcpy.na.NAClassFieldMappings(layer_object, incidents_layer_name)
field_mappings["Name"].mappedFieldName = "Name"
arcpy.na.AddLocations(
    in_network_analysis_layer = layer_object, 
    sub_layer = incidents_layer_name, 
    in_table = intsct_point, 
    field_mappings = field_mappings)

# Solve the closest facility layer and copy the travered source features to the output geodatabase. 
# Use default names for the output feature classes and table. Retrieve the first output, which is the traversed edges.
traversed_edges = arcpy.na.CopyTraversedSourceFeatures(layer_object,
                                                       workspace).getOutput(0)

# Some streets might be traversed by more than one route. Streets traversed by many routes are 
# the most critical emergency routes. Count the number of routes using each street.
edge_frequency = os.path.join(workspace, "Edge_frequency")
arcpy.analysis.Frequency(traversed_edges, 
                         edge_frequency,
                         ["SourceOID", "SourceName"])

#The resulting edge features from CopyTraversedSourceFeatures may include clipped versions of the original street features 
# because the Closest Facility route only traveled across part of the street feature. Select the complete street features 
# from the original street feature class and copy them to output. Get the full path to the network dataset's streets feature 
# class by describing the network dataset.
network_desc = arcpy.Describe(nds)
edge_sources = network_desc.edgeSources
for es in edge_sources:
    if es.name.lower() == "streets":
        streets_source = os.path.join(os.path.dirname(network), es.name)
        break

#Select the relevant streets based on overlap with the results from CopyTraversedSourceFeatures
streets_layer = arcpy.management.MakeFeatureLayer(streets_source,
                                                  "StreetsLayer")
arcpy.management.SelectLayerByLocation(streets_layer,
                                "SHARE_A_LINE_SEGMENT_WITH", traversed_edges)

#Add the frequency information to the output feature class using JoinField
arcpy.management.JoinField(streets_layer, "ObjectID", edge_frequency,
                           "SourceOID", "FREQUENCY")

#Save the selected features to disk
critical_streets = os.path.join(output_gdb, "CriticalStreets")
arcpy.management.CopyFeatures(streets_layer, critical_streets)
#Delete the Frequency field from the street feature class
arcpy.management.DeleteField(streets_layer, "FREQUENCY")

# Delete unnecessary local variables
arcpy.management.Delete(list_pFl)
arcpy.management.Delete(list_pBf)
arcpy.management.Delete(merged_point)
arcpy.management.Delete(merged_polygon)
arcpy.management.Delete(output_summary)


In [ ]:
### SAS - 220606 - v3-2).Closest Facility + Directions

# import system modules
import arcpy
from arcpy import env
import os
import datetime
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
DepartureTime = arcpy.GetParameterAsText(2) # Date (Date & Time)
output_table_xlsx = arcpy.GetParameterAsText(3) # xlsx
output_summary_xlsx = arcpy.GetParameterAsText(4) # xlsx

# define workspace & create feature dataset for SAS feature layers
arcpy.env.workspace = work_dbs
workspace = work_dbs
arcpy.management.CreateFeatureDataset(out_dataset_path = workspace, 
                                      out_name = "SAS",
                                      spatial_reference = arcpy.SpatialReference("WGS 1984")
                                     )

# For each row, print the fields of feature class
fields = ['ID', 'Lat', 'Long', 'Buffer_Distance']
sr = arcpy.SpatialReference("WGS 1984")
with arcpy.da.SearchCursor(loc_csv, fields) as cursor:
    for row in cursor:
        pointNum = 'point_{}'.format(row[0])
        pointGeom_Num = 'pointGeom_{}'.format(row[0])
        pointFlyr_Num = 'pointFlyr_{}'.format(row[0])
        pointBuff_Num = 'pointBuff_{}'.format(row[0])
        pointBuff_Dis = '{} Miles'.format(row[3])
        pointNum = arcpy.Point(row[2],row[1])
        pointGeom_Num = arcpy.PointGeometry(pointNum,sr)
        arcpy.CopyFeatures_management(pointGeom_Num, pointFlyr_Num)
        arcpy.Buffer_analysis(pointFlyr_Num, pointBuff_Num, pointBuff_Dis)

# List all of the polygon feature classes that start with pointFlyr and pointBuff
list_pFl = arcpy.ListFeatureClasses("pointFlyr*", "point")        
list_pBf = arcpy.ListFeatureClasses("pointBuff*", "Polygon")
# merge all of the point and polygon features 
merged_point = os.path.join(workspace, "merged_point")
merged_polygon = os.path.join(workspace, "merged_polygon")
arcpy.Merge_management(list_pFl, merged_point)
arcpy.Merge_management(list_pBf, merged_polygon)

# Copy and join the fields from the csv file
point_merged = os.path.join(workspace, "SAS", "Target_point")
polygon_merged = os.path.join(workspace, "SAS", "Area_polygon")
arcpy.JoinField_management(merged_point, "OBJECTID", 
                           loc_csv, "ID")
arcpy.JoinField_management(merged_polygon, "OBJECTID", 
                           loc_csv, "ID")
arcpy.Copy_management(merged_point, point_merged)
arcpy.Copy_management(merged_polygon, polygon_merged)

# create intersections between the buffer and major roads feature layer
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "SAS", "Road_point")
arcpy.Intersect_analysis([polygon_merged, major_roads], intsct_point, "", "", "point")

# 2. Network Analysis - Closest Facility with arcpy.nax (arcpy.nax cannot create a network analysis)
# set local variables to make closest facility analysis layer and solve it.
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_facilities = point_merged
input_incidents = intsct_point
output_routes = os.path.join(workspace, "SAS", "Route_line")

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a ClosestFacility solver object
closest_facility = arcpy.nax.ClosestFacility(nds)
# Set properties
closest_facility.distanceUnits = arcpy.nax.DistanceUnits.Miles
closest_facility.routeShapeType = arcpy.nax.RouteShapeType.TrueShapeWithMeasures
closest_facility.timeOfDay = datetime.datetime.strptime(DepartureTime, '%m/%d/%Y %I:%M:%S %p')
closest_facility.timeOfDayUsage = arcpy.nax.TimeOfDayUsage.DepartureTime
closest_facility.timeUnits = arcpy.nax.TimeUnits.Minutes
closest_facility.timeZone = arcpy.nax.TimeZoneUsage.UTC
closest_facility.travelDirection = arcpy.nax.TravelDirection.ToFacility
closest_facility.travelMode = travel_mode
closest_facility.defaultTargetFacilityCount = 1
# Load inputs
closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Facilities, input_facilities)
closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Incidents, input_incidents)

# Solve the analysis
result = closest_facility.solve()

# Export the results to a feature class
if result.solveSucceeded:
    result.export(arcpy.nax.ClosestFacilityOutputDataType.Routes, output_routes)
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_routes, output_table_xlsx)

# Execute Summary Statistics and create a table
output_summary = os.path.join(workspace, "Summary_table")
arcpy.Statistics_analysis(
    in_table = output_routes, 
    out_table = output_summary, 
    statistics_fields = [["Total_Minutes", "MEAN"],
                         ["Total_Minutes", "MAX"],
                         ["Total_Minutes", "MIN"],
                         ["Total_Minutes", "MEDIAN"],
                         ["Total_Minutes", "RANGE"],
                         ["Total_Miles", "MEAN"],
                         ["Total_Miles", "MAX"],
                         ["Total_Miles", "MIN"],
                         ["Total_Miles", "MEDIAN"],
                         ["Total_Miles", "RANGE"],
                        ],
    case_field = "Name"
)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_summary, output_summary_xlsx)

# 3. Network Analysis - Make Closest Facility Analysis Layer with arcpy.na
# Create a new network analysis layer with Arcpy.na to calculate the driving directions
result_object = arcpy.na.MakeClosestFacilityAnalysisLayer(
    network_data_source = nds, 
    layer_name = "ClosestTarget",
    travel_mode = "Driving Time", 
    travel_direction = "TO_FACILITIES",
    number_of_facilities_to_find = 1, 
    time_of_day = DepartureTime, 
    time_zone = "UTC", 
    time_of_day_usage = "START_TIME",
    line_shape = "ALONG_NETWORK", 
    generate_directions_on_solve = False
)

# Get the layer object from the result object. The closest facility layer can now be referenced using the layer object.
layer_object = result_object.getOutput(0)

# Get the names of all the sublayers within the closest facility layer.
sublayer_names = arcpy.na.GetNAClassNames(layer_object)
# Stores the layer names that will be used later
facilities_layer_name = sublayer_names["Facilities"]
incidents_layer_name = sublayer_names["Incidents"]

# Laod the targets as Facilities using the default field mappings and search tolerance
arcpy.na.AddLocations(
    in_network_analysis_layer = layer_object,
    sub_layer = facilities_layer_name,
    in_table = point_merged)

# Load the starting major road points as Incidents. Map the Name property from the Name field using field mappings.
field_mappings = arcpy.na.NAClassFieldMappings(layer_object, incidents_layer_name)
field_mappings["Name"].mappedFieldName = "Name"
arcpy.na.AddLocations(
    in_network_analysis_layer = layer_object, 
    sub_layer = incidents_layer_name, 
    in_table = intsct_point, 
    field_mappings = field_mappings)

#Solve the closest facility layer
arcpy.na.Solve(layer_object)

# Save the solved closest facility layer as a layer file on disk
output_layer_file = os.path.join(workspace, "ClosestTarget.lyrx")
layer_object.saveACopy(output_layer_file)

# Generate driving directions in an HTML file
arcpy.na.Directions(in_network_analysis_layer = layer_object,
                    file_type = "HTML", 
                    out_directions_file = "C:/ArcGIS/Projects/Dev_SAS/Direction1.html", 
                    report_units = "Miles",
                    report_time = True,
                    language = "en"
                   )

# Delete unnecessary local variables
arcpy.management.Delete(list_pFl)
arcpy.management.Delete(list_pBf)
arcpy.management.Delete(merged_point)
arcpy.management.Delete(merged_polygon)
arcpy.management.Delete(output_summary)


In [2]:
languages = arcpy.na.ListDirectionsLanguages()
print(languages)

['ar', 'bs', 'ca', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'he', 'hr', 'hu', 'id', 'it', 'ja', 'ko', 'lt', 'lv', 'nb', 'nl', 'pl', 'pt-BR', 'pt-PT', 'ro', 'ru', 'sk', 'sl', 'sr', 'sv', 'th', 'tr', 'uk', 'vi', 'zh-CN', 'zh-HK', 'zh-TW']


In [ ]:
### SAS - 220606 - v3.Closest Facility -1

# import system modules
import arcpy
from arcpy import env
import os
import datetime
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
DepartureTime = arcpy.GetParameterAsText(2) # Date (Date & Time)
output_table_xlsx = arcpy.GetParameterAsText(3) # xlsx
output_summary_xlsx = arcpy.GetParameterAsText(4) # xlsx

# define workspace & create feature dataset for SAS feature layers
arcpy.env.workspace = work_dbs
workspace = work_dbs
arcpy.management.CreateFeatureDataset(out_dataset_path = workspace, 
                                      out_name = "SAS",
                                      spatial_reference = arcpy.SpatialReference("WGS 1984")
                                     )

# For each row, print the fields of feature class
fields = ['ID', 'Lat', 'Long', 'Buffer_Distance']
sr = arcpy.SpatialReference("WGS 1984")
with arcpy.da.SearchCursor(loc_csv, fields) as cursor:
    for row in cursor:
        pointNum = 'point_{}'.format(row[0])
        pointGeom_Num = 'pointGeom_{}'.format(row[0])
        pointFlyr_Num = 'pointFlyr_{}'.format(row[0])
        pointBuff_Num = 'pointBuff_{}'.format(row[0])
        pointBuff_Dis = '{} Miles'.format(row[3])
        pointNum = arcpy.Point(row[2],row[1])
        pointGeom_Num = arcpy.PointGeometry(pointNum,sr)
        arcpy.CopyFeatures_management(pointGeom_Num, pointFlyr_Num)
        arcpy.Buffer_analysis(pointFlyr_Num, pointBuff_Num, pointBuff_Dis)

# List all of the polygon feature classes that start with pointFlyr and pointBuff
list_pFl = arcpy.ListFeatureClasses("pointFlyr*", "point")        
list_pBf = arcpy.ListFeatureClasses("pointBuff*", "Polygon")
# merge all of the point and polygon features 
merged_point = os.path.join(workspace, "merged_point")
merged_polygon = os.path.join(workspace, "merged_polygon")
arcpy.Merge_management(list_pFl, merged_point)
arcpy.Merge_management(list_pBf, merged_polygon)

# Copy and join the fields from the csv file
point_merged = os.path.join(workspace, "SAS", "Target_point")
polygon_merged = os.path.join(workspace, "SAS", "Area_polygon")
arcpy.JoinField_management(merged_point, "OBJECTID", 
                           loc_csv, "ID")
arcpy.JoinField_management(merged_polygon, "OBJECTID", 
                           loc_csv, "ID")
arcpy.Copy_management(merged_point, point_merged)
arcpy.Copy_management(merged_polygon, polygon_merged)

# create intersections between the buffer and major roads feature layer
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "SAS", "Road_point")
arcpy.Intersect_analysis([polygon_merged, major_roads], intsct_point, "", "", "point")

# 2. Network Analysis - Closest Facility
# set local variables to make closest facility analysis layer and solve it.
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_facilities = point_merged
input_incidents = intsct_point
output_routes = os.path.join(workspace, "SAS", "Route_line")

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a ClosestFacility solver object
closest_facility = arcpy.nax.ClosestFacility(nds)
# Set properties
closest_facility.distanceUnits = arcpy.nax.DistanceUnits.Miles
closest_facility.routeShapeType = arcpy.nax.RouteShapeType.TrueShapeWithMeasures
closest_facility.timeOfDay = datetime.datetime.strptime(DepartureTime, '%m/%d/%Y %I:%M:%S %p')
closest_facility.timeOfDayUsage = arcpy.nax.TimeOfDayUsage.DepartureTime
closest_facility.timeUnits = arcpy.nax.TimeUnits.Minutes
closest_facility.timeZone = arcpy.nax.TimeZoneUsage.UTC
closest_facility.travelDirection = arcpy.nax.TravelDirection.ToFacility
closest_facility.travelMode = travel_mode
closest_facility.defaultTargetFacilityCount = 1
# Load inputs
closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Facilities, input_facilities)
closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Incidents, input_incidents)

# Solve the analysis
result = closest_facility.solve()

# Export the results to a feature class
if result.solveSucceeded:
    result.export(arcpy.nax.ClosestFacilityOutputDataType.Routes, output_routes)
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_routes, output_table_xlsx)

# Execute Summary Statistics and create a table
output_summary = os.path.join(workspace, "Summary_table")
arcpy.Statistics_analysis(
    in_table = output_routes, 
    out_table = output_summary, 
    statistics_fields = [["Total_Minutes", "MEAN"],
                         ["Total_Minutes", "MAX"],
                         ["Total_Minutes", "MIN"],
                         ["Total_Minutes", "MEDIAN"],
                         ["Total_Minutes", "RANGE"],
                         ["Total_Miles", "MEAN"],
                         ["Total_Miles", "MAX"],
                         ["Total_Miles", "MIN"],
                         ["Total_Miles", "MEDIAN"],
                         ["Total_Miles", "RANGE"],
                        ],
    case_field = "Name"
)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_summary, output_summary_xlsx)

# Delete unnecessary local variables
arcpy.management.Delete(list_pFl)
arcpy.management.Delete(list_pBf)
arcpy.management.Delete(merged_point)
arcpy.management.Delete(merged_polygon)
arcpy.management.Delete(output_summary)


In [ ]:
### SAS - 220603 - v2.Make Closest Facility Analysis Layer

# import system modules
import arcpy
from arcpy import env
import os
import datetime
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
DepartureTime = arcpy.GetParameterAsText(2) # Date (Date & Time)
output_table_xlsx = arcpy.GetParameterAsText(3) # xlsx
output_summary_xlsx = arcpy.GetParameterAsText(4) # xlsx

# define workspace & create feature dataset for SAS feature layers
arcpy.env.workspace = work_dbs
workspace = work_dbs
arcpy.management.CreateFeatureDataset(out_dataset_path = workspace, 
                                      out_name = "SAS",
                                      spatial_reference = arcpy.SpatialReference("WGS 1984")
                                     )

# For each row, print the fields of feature class
fields = ['ID', 'Lat', 'Long', 'Buffer_Distance']
sr = arcpy.SpatialReference("WGS 1984")
with arcpy.da.SearchCursor(loc_csv, fields) as cursor:
    for row in cursor:
        pointNum = 'point_{}'.format(row[0])
        pointGeom_Num = 'pointGeom_{}'.format(row[0])
        pointFlyr_Num = 'pointFlyr_{}'.format(row[0])
        pointBuff_Num = 'pointBuff_{}'.format(row[0])
        pointBuff_Dis = '{} Miles'.format(row[3])
        pointNum = arcpy.Point(row[2],row[1])
        pointGeom_Num = arcpy.PointGeometry(pointNum,sr)
        arcpy.CopyFeatures_management(pointGeom_Num, pointFlyr_Num)
        arcpy.Buffer_analysis(pointFlyr_Num, pointBuff_Num, pointBuff_Dis)

# List all of the polygon feature classes that start with pointFlyr and pointBuff
list_pFl = arcpy.ListFeatureClasses("pointFlyr*", "point")        
list_pBf = arcpy.ListFeatureClasses("pointBuff*", "Polygon")
# merge all of the point and polygon features 
merged_point = os.path.join(workspace, "merged_point")
merged_polygon = os.path.join(workspace, "merged_polygon")
arcpy.Merge_management(list_pFl, merged_point)
arcpy.Merge_management(list_pBf, merged_polygon)

# Copy and join the fields from the csv file
point_merged = os.path.join(workspace, "SAS", "Target_point")
polygon_merged = os.path.join(workspace, "SAS", "Area_polygon")
arcpy.JoinField_management(merged_point, "OBJECTID", 
                           loc_csv, "ID")
arcpy.JoinField_management(merged_polygon, "OBJECTID", 
                           loc_csv, "ID")
arcpy.Copy_management(merged_point, point_merged)
arcpy.Copy_management(merged_polygon, polygon_merged)

# create intersections between the buffer and major roads feature layer
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "SAS", "Road_point")
arcpy.Intersect_analysis([polygon_merged, major_roads], intsct_point, "", "", "point")

# 2-1. Network Analysis - Closest Facility Analysis
# set local variables to make closest facility analysis layer and solve it.
network = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
facilities = point_merged
incidents = intsct_point
layer_name = "ClosestFacility"
travel_mode = "Driving Time"
output_layer_file = os.path.join(workspace, layer_name + ".lyrx")

# Create a new closest facility analysis layer
result_object = arcpy.na.MakeClosestFacilityAnalysisLayer(network, layer_name, 
                                                          travel_mode, 
                                                          travel_direction = "TO_FACILITIES",
                                                          number_of_facilities_to_find = 1, 
                                                          time_of_day = DepartureTime, 
                                                          time_zone = "UTC", 
                                                          time_of_day_usage = "START_TIME",
                                                          line_shape = "ALONG_NETWORK", 
                                                          generate_directions_on_solve = False
                                                         )

# Get the layer object from the result object. The closest facility layer can now be referenced using the layer object.
layer_object = result_object.getOutput(0)

# Get the names of all the sublayers within the closest facility layer.
sublayer_names = arcpy.na.GetNAClassNames(layer_object)
# Stores the layer names that we will use later
facilities_layer_name = sublayer_names["Facilities"]
incidents_layer_name = sublayer_names["Incidents"]

# Load the Target Locations as Facilities using the default field mappings and search tolerance
arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities, "", "")
    
# Load the Road Points as Incidents. Map the Name property from the Name field using field mappings
field_mappings = arcpy.na.NAClassFieldMappings(layer_object, incidents_layer_name)
field_mappings["Name"].mappedFieldName = "Name"
arcpy.na.AddLocations(layer_object, incidents_layer_name, incidents, field_mappings, "")

#Solve the closest facility layer
arcpy.na.Solve(layer_object)

# Save the solved closest facility layer as a layer file on disk
layer_object.saveACopy(output_layer_file)

# Export from feature class table to excel
# arcpy.TableToExcel_conversion(output_layer_file, output_table_xlsx)

# Delete unnecessary local variables
arcpy.management.Delete(list_pFl)
arcpy.management.Delete(list_pBf)
arcpy.management.Delete(merged_point)
arcpy.management.Delete(merged_polygon)
#arcpy.management.Delete(output_summary)

In [ ]:
### SAS - 220602 - v1.OD-CostMatrix

# import system modules
import arcpy
from arcpy import env
import os
import datetime
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
DepartureTime = arcpy.GetParameterAsText(2) # Date (Date & Time)
output_table_xlsx = arcpy.GetParameterAsText(3) # xlsx
output_summary_xlsx = arcpy.GetParameterAsText(4) # xlsx

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs
arcpy.management.CreateFeatureDataset(out_dataset_path = workspace, 
                                      out_name = "SAS",
                                      spatial_reference = arcpy.SpatialReference("WGS 1984")
                                     )

# For each row, print the fields of feature class
fields = ['ID', 'Lat', 'Long', 'Buffer_Distance']
sr = arcpy.SpatialReference("WGS 1984")
with arcpy.da.SearchCursor(loc_csv, fields) as cursor:
    for row in cursor:
        pointNum = 'point_{}'.format(row[0])
        pointGeom_Num = 'pointGeom_{}'.format(row[0])
        pointFlyr_Num = 'pointFlyr_{}'.format(row[0])
        pointBuff_Num = 'pointBuff_{}'.format(row[0])
        pointBuff_Dis = '{} Miles'.format(row[3])
        pointNum = arcpy.Point(row[2],row[1])
        pointGeom_Num = arcpy.PointGeometry(pointNum,sr)
        arcpy.CopyFeatures_management(pointGeom_Num, pointFlyr_Num)
        arcpy.Buffer_analysis(pointFlyr_Num, pointBuff_Num, pointBuff_Dis)

# List all of the polygon feature classes that start with pointFlyr and pointBuff
list_pFl = arcpy.ListFeatureClasses("pointFlyr*", "point")        
list_pBf = arcpy.ListFeatureClasses("pointBuff*", "Polygon")

# merge all of the point and polygon features 
merged_point = os.path.join(workspace, "merged_point")
merged_polygon = os.path.join(workspace, "merged_polygon")
arcpy.Merge_management(list_pFl, merged_point)
arcpy.Merge_management(list_pBf, merged_polygon)

# Copy and join the fields from the csv file
point_merged = os.path.join(workspace, "SAS", "Target_point")
polygon_merged = os.path.join(workspace, "SAS", "Area_polygon")
arcpy.JoinField_management(merged_point, "OBJECTID", 
                           loc_csv, "ID")
arcpy.JoinField_management(merged_polygon, "OBJECTID", 
                           loc_csv, "ID")
arcpy.Copy_management(merged_point, point_merged)
arcpy.Copy_management(merged_polygon, polygon_merged)

# create intersections between the buffer and major roads feature layer
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "SAS", "Road_point")
arcpy.Intersect_analysis([polygon_merged, major_roads], intsct_point, "", "", "point")

# set local variables to perform origin destination cost matrix
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origin = intsct_point
input_destination = merged_point

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultDestinationCount = 1
odcm.distanceUnits = arcpy.nax.DistanceUnits.Miles
odcm.lineShapeType = arcpy.nax.LineShapeType.StraightLine
odcm.timeOfDay = datetime.datetime.strptime(DepartureTime, '%m/%d/%Y %I:%M:%S %p')
odcm.timeZone = arcpy.nax.TimeZoneUsage.UTC
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origin)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destination)
# Solve the analysis
result = odcm.solve()
# Set Time of Day / Is Time of Day in UTC

# Export the results to a feature class
output_lines = os.path.join(workspace, "SAS", "ODCM_line")
result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)

# Table join with the point feuatres 
output_point = os.path.join(workspace, "SAS", "ODCM_point")
arcpy.JoinField_management(intsct_point, "OBJECTID", 
                           output_lines, "OBJECTID")
arcpy.Copy_management(intsct_point, output_point)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_point, output_table_xlsx)

# Execute Summary Statistics and create a table
output_summary = os.path.join(workspace, "Summary_table")
arcpy.Statistics_analysis(
    in_table = output_point, 
    out_table = output_summary, 
    statistics_fields = [["Total_Time", "MEAN"],
                         ["Total_Time", "MAX"],
                         ["Total_Time", "MIN"],
                         ["Total_Time", "MEDIAN"],
                         ["Total_Time", "RANGE"],
                         ["Total_Distance", "MEAN"],
                         ["Total_Distance", "MAX"],
                         ["Total_Distance", "MIN"],
                         ["Total_Distance", "MEDIAN"],
                         ["Total_Distance", "RANGE"],
                        ],
    case_field = "Name"
)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_summary, output_summary_xlsx)

# Delete unnecessary local variables
arcpy.management.Delete(list_pFl)
arcpy.management.Delete(list_pBf)
arcpy.management.Delete(merged_point)
arcpy.management.Delete(merged_polygon)
arcpy.management.Delete(output_summary)


In [ ]:
### SAS - 220601 - Worked

# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
output_table_xlsx = arcpy.GetParameterAsText(2) # xlsx
output_summary_xlsx = arcpy.GetParameterAsText(3) # xlsx

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

# For each row, print the fields of feature class
fields = ['ID', 'Lat', 'Long', 'Buffer_Distance']
sr = arcpy.SpatialReference("WGS 1984")
with arcpy.da.SearchCursor(loc_csv, fields) as cursor:
    for row in cursor:
        pointNum = 'point_{}'.format(row[0])
        pointGeom_Num = 'pointGeom_{}'.format(row[0])
        pointFlyr_Num = 'pointFlyr_{}'.format(row[0])
        pointBuff_Num = 'pointBuff_{}'.format(row[0])
        pointBuff_Dis = '{} Miles'.format(row[3])
        pointNum = arcpy.Point(row[2],row[1])
        pointGeom_Num = arcpy.PointGeometry(pointNum,sr)
        arcpy.CopyFeatures_management(pointGeom_Num, pointFlyr_Num)
        arcpy.Buffer_analysis(pointFlyr_Num, pointBuff_Num, pointBuff_Dis)

# List all of the polygon feature classes that start with pointFlyr and pointBuff
list_pFl = arcpy.ListFeatureClasses("pointFlyr*", "point")        
list_pBf = arcpy.ListFeatureClasses("pointBuff*", "Polygon")

# merge all of the point and polygon features 
merged_point = os.path.join(workspace, "merged_point")
merged_polygon = os.path.join(workspace, "merged_polygon")
arcpy.Merge_management(list_pFl, merged_point)
arcpy.Merge_management(list_pBf, merged_polygon)

# Copy and join the fields from the csv file
point_merged = os.path.join(workspace, "point_merged")
polygon_merged = os.path.join(workspace, "polygon_merged")
arcpy.JoinField_management(merged_point, "OBJECTID", 
                           loc_csv, "ID")
arcpy.JoinField_management(merged_polygon, "OBJECTID", 
                           loc_csv, "ID")
arcpy.Copy_management(merged_point, point_merged)
arcpy.Copy_management(merged_polygon, polygon_merged)

# create intersections between the buffer and major roads feature layer
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "intsct_point")
arcpy.Intersect_analysis([polygon_merged, major_roads], intsct_point, "", "", "point")

# set local variables to perform origin destination cost matrix
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origin = intsct_point
input_destination = merged_point

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultDestinationCount = 1
odcm.distanceUnits = arcpy.nax.DistanceUnits.Miles
odcm.lineShapeType = arcpy.nax.LineShapeType.StraightLine
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origin)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destination)
# Solve the analysis
result = odcm.solve()
# Set Time of Day / Is Time of Day in UTC

# Export the results to a feature class
output_lines = os.path.join(workspace, "output_lines")
result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)

# Table join with the point feuatres 
output_point = os.path.join(workspace, "output_point")
arcpy.JoinField_management(intsct_point, "OBJECTID", 
                           output_lines, "OBJECTID")
arcpy.Copy_management(intsct_point, output_point)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_point, output_table_xlsx)

# Execute Summary Statistics and create a table
output_summary = os.path.join(workspace, "output_summary")
arcpy.Statistics_analysis(
    in_table = output_point, 
    out_table = output_summary, 
    statistics_fields = [["Total_Time", "MEAN"],
                         ["Total_Time", "MAX"],
                         ["Total_Time", "MIN"],
                         ["Total_Time", "MEDIAN"],
                         ["Total_Time", "RANGE"],
                         ["Total_Distance", "MEAN"],
                         ["Total_Distance", "MAX"],
                         ["Total_Distance", "MIN"],
                         ["Total_Distance", "MEDIAN"],
                         ["Total_Distance", "RANGE"],
                        ],
    case_field = "Name"
)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_summary, output_summary_xlsx)

# Delete unnecessary local variables
arcpy.management.Delete(list_pFl)
arcpy.management.Delete(list_pBf)
arcpy.management.Delete(merged_point)
arcpy.management.Delete(merged_polygon)


In [ ]:
### 220531 Site Access Score | Round 3

# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

###################################################################################################
# 1. One Site to Multiple Sites

# Change the in_lat, in_long, and buffer_dist parameters to the csv parameters with the columns
# 1) Use XYTableToPoint Tool for in_lat and in_long
# 2) Set the distance column to a new parameter and use it instead of buffer_dist in Buffer_Analysis tool

###################################################################################################

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
location_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
# in_lat = arcpy.GetParameterAsText(1) # Double, Y
# in_long = arcpy.GetParameterAsText(2) # Double, X
point_target = arcpy.GetParameterAsText(2) # feature layer - Target Point
# point_buffer = arcpy.GetParameterAsText(4) # feature layer - Buffer Area
buffer_dist = arcpy.GetParameterAsText(3) # Linear unit - Buffer Distance
output_point = arcpy.GetParameterAsText(4) # feature layer
output_point_xlsx = arcpy.GetParameterAsText(5) # xlsx
output_table_xlsx = arcpy.GetParameterAsText(6) # xlsx

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

# Create point features for the target locations
xy_point = os.path.join(workspace, "point")
arcpy.management.XYTableToPoint(
    in_table = location_csv, 
    out_feature_class = xy_point,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_target, point_buffer, buffer_dist)

# create intersections between the buffer and major roads feature layer
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "intsct_point")
arcpy.Intersect_analysis([point_buffer, major_roads], intsct_point, "", "", "point")

# set local variables to perform origin destination cost matrix
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origin = intsct_point
input_destination = point_target

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultDestinationCount = 1
odcm.distanceUnits = arcpy.nax.DistanceUnits.Miles
odcm.lineShapeType = arcpy.nax.LineShapeType.StraightLine
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origin)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destination)
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
output_lines = os.path.join(workspace, "output_lines")
result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)

# Table join with the point feuatres 
field = "OBJECTID"
arcpy.JoinField_management(intsct_point, field, 
                           output_lines, field)
arcpy.Copy_management(intsct_point, output_point)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_point, output_point_xlsx)

# Execute Summary Statistics and create a table
output_table = os.path.join(workspace, "output_table")
arcpy.Statistics_analysis(
    in_table = output_lines, 
    out_table = output_table, 
    statistics_fields = [["Total_Time", "MEAN"],
                         ["Total_Time", "MAX"],
                         ["Total_Time", "MIN"],
                         ["Total_Time", "MEDIAN"],
                         ["Total_Time", "RANGE"],
                         ["Total_Distance", "MEAN"],
                         ["Total_Distance", "MAX"],
                         ["Total_Distance", "MIN"],
                         ["Total_Distance", "MEDIAN"],
                         ["Total_Distance", "RANGE"],
                        ])

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_table, output_table_xlsx)

# Use ORDER BY sql clause to sort field values
fields = ['OID@', 'Total_Time', 'Total_Distance']
with arcpy.da.UpdateCursor(
    in_table = output_lines, 
    field_names = fields, 
    sql_clause=(None, 'ORDER BY Total_Time DESC')) as cursor:
    for row in cursor:
        row[1] = round(float(row[1]), 1)
        row[2] = round(float(row[2]), 2)
        cursor.updateRow(row)
        arcpy.AddMessage('ID: {}, Drive Time: {} Minutes, Drive Distance: {} Miles'.format(row[0], row[1], row[2]))
    
# Delete unnecessary data
arcpy.management.Delete(intsct_point)
arcpy.management.Delete(output_lines)
arcpy.management.Delete(output_table)


In [ ]:
# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
xy_point = arcpy.GetParameterAsText(2) # Feature Layer

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

### 1-2. Current Version
# Create point features
arcpy.management.XYTableToPoint(
    in_table = loc_csv, 
    out_feature_class = xy_point,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))


In [ ]:
# Reference 1

mxd = arcpy.mapping.MapDocument("Current")         
df = arcpy.mapping.ListDataFrames(mxd)[0]         
fc = 'Database Connections/TDPMode.sde/KWAGISMAIN.DBO.Parcels'         
field = "OBJECTID"         
field1 = "LGA"         
field2 = "District"         
field3 = "Block_No"         
field4 = "Plot_No"         
rows = arcpy.SearchCursor(fc)         
row = rows.next()         
while row:             
    val = row.getValue(field)             
    val1 = row.getValue(field1)             
    val2 = row.getValue(field2)             
    val3 = row.getValue(field3)             
    val4 = row.getValue(field4)             
    whereClause = '"OBJECTID"' + " = '" + str(val) + "'"             
    outName = "Block_" + str(val3) + "_Plot_" + str(val4) + "_" + str(val2) + "_Area_of_" + str(val1) + "_LGA" + ".pdf"             
    path = "C:/Users/Administrator/Documents/ArcGIS/Default.gdb/"      
    
arcpy.SelectLayerByAttribute_management("Parcels", "NEW_SELECTION", whereClause)             

arcpy.Buffer_analysis ("Parcels", "DataDrivenPage_Buffer", "14 meters", "FULL", "ROUND", "NONE")             

layer_list = "Lines","Parcels","Access_Road_Graphics","Points","Lines_Split"             

for layer in layer_list:                 
    arcpy.SelectLayerByLocation_management(layer, "COMPLETELY_WITHIN", "DataDrivenPage_Buffer", "", "NEW_SELECTION")             
    exportLayer = "Points", "Lines", "Parcels","Lines_Split","Access_Road_Graphics"             

for layer in exportLayer:                 
    outFC = path + layer + "_New"                  
    arcpy.Clip_analysis(layer,"DataDrivenPage_Buffer",outFC)                 

arcpy.SelectLayerByAttribute_management(layer, "CLEAR_SELECTION")             
mxd = arcpy.mapping.MapDocument("current")          
lyr = arcpy.mapping.ListLayers(mxd, "Lines_New")[0]             

for lblClass in lyr.labelClasses:                 
    lblClass.SQLQuery = '"ParcelID"' + "=" + str(val)                  
    arcpy.RefreshActiveView()             

arcpy.SelectLayerByAttribute_management("DataDrivenPage_Buffer", "NEW_SELECTION", "OBJECTID = 1")             
df.zoomToSelectedFeatures()             
mxd = arcpy.mapping.MapDocument("Current")             

for df in arcpy.mapping.ListDataFrames(mxd):                 
    df.rotation = 0             
    if df.scale <= 400:                  
        df.scale = 500             
        if df.scale > 400 and df.scale < 1000:                 
            df.scale = 1000                 
            else:                  
                df.scale = 2000             
                
arcpy.mapping.ExportToPDF(mxd,r"C:\STATE\\Batch\\" + "TDP_For_" + outName)             
row = rows.next()

In [ ]:
# Reference 2

print ('Comparing Pending against Confirmed...')
##Select ScotWind_Applications_Confirmed by location within 5km of the selected row of ScotWind_Applications_Pending##
with arcpy.da.SearchCursor("ScotWind_Applications_Pending", ["OBJECTID", "ScotWind_Applications_SMPJoin_AppID", "Rank", "SHAPE@AREA"], sql_clause=(None, "ORDER BY Rank DESC")) as cursor:
    for row in cursor:
        ##Select the current row in ScotWind_Applications_Pending##
        arcpy.SelectLayerByAttribute_management ("ScotWind_Applications_Pending","NEW_SELECTION", "OBJECTID = {}".format(row[0]))
        
        ##Create a Layer of all Confirmed Applications which is updated on each iteration##
        arcpy.MakeFeatureLayer_management (ScotWind_Applications_Scoring, "ScotWind_Applications_Confirmed")
        arcpy.management.SelectLayerByAttribute ("ScotWind_Applications_Confirmed", "NEW_SELECTION", "Status = 'Confirmed'", None)

        area_sum = 0
        with arcpy.da.SearchCursor("ScotWind_Applications_Confirmed", ["SHAPE@AREA"]) as cursor2:
           for row2 in cursor2:
                area_sum = area_sum + row2[0]
                                                     
        print (area_sum)

        universal_sum = area_sum + row[3]
        
        ##Select all Confirmed Applications on each iteration that are within 5km of the 
        arcpy.management.SelectLayerByLocation("ScotWind_Applications_Confirmed", "INTERSECT", "ScotWind_Applications_Pending", "5 Kilometers", "SUBSET_SELECTION", "NOT_INVERT")

        featurecount = int(arcpy.GetCount_management("ScotWind_Applications_Confirmed").getOutput(0))

        print (u'{0}, {1}, {2}, {3}'.format(row[0], row[1], row[2], row[3]))

        print (featurecount)

        print (universal_sum)
        
        codeblock5 = """def Reclass(Status):
            if universal_sum > 4500000000:
                return 'Capped Out'
            elif featurecount== 0:
                return 'Confirmed'
            else:
                return 'Defeated'"""
                
        arcpy.management.CalculateField("ScotWind_Applications_Pending", "Status", "Reclass(!Status!)", "PYTHON3", codeblock5)

        arcpy.Delete_management("ScotWind_Applications_Confirmed")‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍

In [ ]:
### 220526 Site Access Score Tool | 2nd Round

# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
in_lat = arcpy.GetParameterAsText(1) # Double, Y
in_long = arcpy.GetParameterAsText(2) # Double, X
point_target = arcpy.GetParameterAsText(3) # feature layer - Target Point
point_buffer = arcpy.GetParameterAsText(4) # feature layer - Buffer Area
buffer_dist = arcpy.GetParameterAsText(5) # Linear unit - Buffer Distance
output_point = arcpy.GetParameterAsText(6) # feature layer
output_point_xlsx = arcpy.GetParameterAsText(7) # xlsx
output_table_xlsx = arcpy.GetParameterAsText(8) # xlsx

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

# create a point with spatial reference
point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)
# create a point feature class
arcpy.CopyFeatures_management(point_geom, point_target)

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_target, point_buffer, buffer_dist)

# create intersections between the buffer and major roads feature layer
  # Set local variable
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "intsct_point")
arcpy.Intersect_analysis([point_buffer, major_roads], intsct_point, "", "", "point")

# set local variables to perform origin destination cost matrix
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origin = intsct_point
input_destination = point_target

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultDestinationCount = 1
odcm.distanceUnits = arcpy.nax.DistanceUnits.Miles
odcm.lineShapeType = arcpy.nax.LineShapeType.StraightLine
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origin)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destination)
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
output_lines = os.path.join(workspace, "output_lines")
result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)

# Table join with the point feuatres 
field = "OBJECTID"
arcpy.JoinField_management(intsct_point, field, 
                           output_lines, field)
arcpy.Copy_management(intsct_point, output_point)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_point, output_point_xlsx)

# Execute Summary Statistics and create a table
output_table = os.path.join(workspace, "output_table")
arcpy.Statistics_analysis(
    in_table = output_lines, 
    out_table = output_table, 
    statistics_fields = [["Total_Time", "MEAN"],
                         ["Total_Time", "MAX"],
                         ["Total_Time", "MIN"],
                         ["Total_Time", "MEDIAN"],
                         ["Total_Time", "RANGE"],
                         ["Total_Distance", "MEAN"],
                         ["Total_Distance", "MAX"],
                         ["Total_Distance", "MIN"],
                         ["Total_Distance", "MEDIAN"],
                         ["Total_Distance", "RANGE"],
                        ])

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_table, output_table_xlsx)

# Use ORDER BY sql clause to sort field values
fields = ['OID@', 'Total_Time', 'Total_Distance']
with arcpy.da.UpdateCursor(
    in_table = output_lines, 
    field_names = fields, 
    sql_clause=(None, 'ORDER BY Total_Time DESC')) as cursor:
    for row in cursor:
        row[1] = round(float(row[1]), 1)
        row[2] = round(float(row[2]), 2)
        cursor.updateRow(row)
        arcpy.AddMessage('ID: {}, Drive Time: {} Minutes, Drive Distance: {} Miles'.format(row[0], row[1], row[2]))
    
# Delete unnecessary data
arcpy.management.Delete(intsct_point)
arcpy.management.Delete(output_lines)
arcpy.management.Delete(output_table)
